In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Vertex AI Gen AI 評估服務來評估您的 ADK Agent

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> 在 Colab 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluating_adk_agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/evaluating_adk_agent.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>分享到：</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_adk_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| 作者 |
| --- |
| [Ivan Nardini](https://github.com/inardini) |

## 概述

Agent Development Kit (簡稱 ADK) 是一個靈活且模組化的開源框架，用於開發和部署 AI Agent。雖然 ADK 有自己的評估模組，但使用 Vertex AI Gen AI 評估服務可以提供一套經過品質控管且可解釋的方法和指標工具組，用於評估任何生成式模型或應用程式（包括 Agent），並根據您自己的評估標準來進行基準測試。

**本教學示範如何使用 Vertex AI Gen AI 評估服務來評估 ADK Agent。**

**執行的步驟包括：**

* 使用 ADK 建立本地端 Agent
* 準備 Agent 評估資料集
* 單一工具使用評估
* 軌跡 (Trajectory) 評估
* 回應評估

> **重點說明：** 這個教學涵蓋了 Agent 評估的三個主要層面：工具選擇、執行流程，以及最終輸出品質。

## 開始使用

### 安裝 Google Gen AI SDK 和其他必要套件

> **重點說明：** 使用 --upgrade 確保安裝最新版本，--quiet 減少輸出訊息。

In [ ]:
%pip install --upgrade --quiet 'google-adk'
%pip install --upgrade --quiet 'google-cloud-aiplatform[evaluation]'

### 驗證您的 notebook 環境（僅限 Colab）

如果您在 Google Colab 上執行此 notebook，請執行下方的 cell 來驗證您的環境。

> **重點說明：** Colab 需要額外的驗證步驟來存取 Google Cloud 資源。

In [ ]:
import sys

# 檢查是否在 Colab 環境中執行
# 重點：只有在 Colab 環境才需要進行額外的驗證
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 設定 Google Cloud 專案資訊

要開始使用 Vertex AI，您必須擁有現有的 Google Cloud 專案並[啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

深入了解[設定專案和開發環境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

> **重點說明：** 確保已啟用 Vertex AI API 並建立 Cloud Storage bucket 用於儲存評估結果。

In [ ]:
# 如果使用者沒有提供專案 ID，則使用環境變數
import os

import vertexai

# 重點：請將 [your-project-id] 替換為您的實際 GCP 專案 ID
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# 設定 GCP 區域，預設為美國中部
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# 設定 Cloud Storage bucket 名稱
# 重點：bucket 名稱必須是全域唯一的
BUCKET_NAME = "[your-bucket-name]"  # @param {type: "string", placeholder: "[your-bucket-name]", isTemplate: true}
BUCKET_URI = f"gs://{BUCKET_NAME}"

# 建立 Cloud Storage bucket
# 重點：-l 參數指定 bucket 的位置
!gsutil mb -l {LOCATION} {BUCKET_URI}

# 設定環境變數
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# 設定實驗名稱
# 重點：實驗用於組織和追蹤多次評估執行
EXPERIMENT_NAME = "evaluate-adk-agent"  # @param {type:"string"}

# 初始化 Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)

## 匯入函式庫

匯入教學所需的函式庫。

> **重點說明：** 包含 ADK 核心元件、評估工具、資料處理和視覺化函式庫。

In [ ]:
import json
import asyncio

# 一般用途
import random
import string
from typing import Any

from IPython.display import HTML, Markdown, display

# 使用 ADK 建立 Agent
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# 評估 Agent
from google.cloud import aiplatform
from google.genai import types
import pandas as pd
import plotly.graph_objects as go
# Vertex AI 評估工具
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,  # 點對點評估指標
    PointwiseMetricPromptTemplate,  # 自訂評估提示模板
    TrajectorySingleToolUse,  # 單一工具使用評估
)

## 定義輔助函式

初始化一組輔助函式來列印教學結果。

> **重點說明：** 這些函式用於格式化輸出、解析 Agent 回應、顯示評估報告和視覺化結果。

In [ ]:
def get_id(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def parse_adk_output_to_dictionary(events: list[Event], *, as_json: bool = False):
    """
    Parse ADK event output into a structured dictionary format,
    with the predicted trajectory dumped as a JSON string.

    """

    final_response = ""
    trajectory = []

    for event in events:
        if not getattr(event, "content", None) or not getattr(event.content, "parts", None):
            continue
        for part in event.content.parts:
            if getattr(part, "function_call", None):
                info = {
                    "tool_name": part.function_call.name,
                    "tool_input": dict(part.function_call.args),
                }
                if info not in trajectory:
                    trajectory.append(info)
            if event.content.role == "model" and getattr(part, "text", None):
                final_response = part.text.strip()

    if as_json:
        trajectory_out = json.dumps(trajectory)
    else:
        trajectory_out = trajectory

    return {"response": final_response, "predicted_trajectory": trajectory_out}


def format_output_as_markdown(output: dict) -> str:
    """Convert the output dictionary to a formatted markdown string."""
    markdown = "### AI Response\n" + output["response"] + "\n\n"
    if output["predicted_trajectory"]:
        markdown += "### Function Calls\n"
        for call in output["predicted_trajectory"]:
            markdown += f"- **Function**: `{call['tool_name']}`\n"
            markdown += "  - **Arguments**\n"
            for key, value in call["tool_input"].items():
                markdown += f"    - `{key}`: `{value}`\n"
    return markdown


def display_eval_report(eval_result: pd.DataFrame) -> None:
    """Display the evaluation results."""
    display(Markdown("### Summary Metrics"))
    display(
        pd.DataFrame(
            eval_result.summary_metrics.items(), columns=["metric", "value"]
        )
    )
    if getattr(eval_result, "metrics_table", None) is not None:
        display(Markdown("### Row‑wise Metrics"))
        display(eval_result.metrics_table.head())


def display_drilldown(row: pd.Series) -> None:
    """Displays a drill-down view for trajectory data within a row."""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"

    if not (
        isinstance(row["predicted_trajectory"], list)
        and isinstance(row["reference_trajectory"], list)
    ):
        return

    for predicted_trajectory, reference_trajectory in zip(
        row["predicted_trajectory"], row["reference_trajectory"]
    ):
        display(
            HTML(
                f"<h3>Tool Names:</h3><div style='{style}'>{predicted_trajectory['tool_name'], reference_trajectory['tool_name']}</div>"
            )
        )

        if not (
            isinstance(predicted_trajectory.get("tool_input"), dict)
            and isinstance(reference_trajectory.get("tool_input"), dict)
        ):
            continue

        for tool_input_key in predicted_trajectory["tool_input"]:
            print("Tool Input Key: ", tool_input_key)

            if tool_input_key in reference_trajectory["tool_input"]:
                print(
                    "Tool Values: ",
                    predicted_trajectory["tool_input"][tool_input_key],
                    reference_trajectory["tool_input"][tool_input_key],
                )
            else:
                print(
                    "Tool Values: ",
                    predicted_trajectory["tool_input"][tool_input_key],
                    "N/A",
                )
        print("\n")
    display(HTML("<hr>"))


def display_dataframe_rows(
    df: pd.DataFrame,
    columns: list[str] | None = None,
    num_rows: int = 3,
    display_drilldown: bool = False,
) -> None:
    """Displays a subset of rows from a DataFrame, optionally including a drill-down view."""

    if columns:
        df = df[columns]

    base_style = "font-family: monospace; font-size: 14px; white-space: pre-wrap; width: auto; overflow-x: auto;"
    header_style = base_style + "font-weight: bold;"

    for _, row in df.head(num_rows).iterrows():
        for column in df.columns:
            display(
                HTML(
                    f"<span style='{header_style}'>{column.replace('_', ' ').title()}: </span>"
                )
            )
            display(HTML(f"<span style='{base_style}'>{row[column]}</span><br>"))

        display(HTML("<hr>"))

        if (
            display_drilldown
            and "predicted_trajectory" in df.columns
            and "reference_trajectory" in df.columns
        ):
            display_drilldown(row)


def plot_bar_plot(
    eval_result: pd.DataFrame, title: str, metrics: list[str] = None
) -> None:
    fig = go.Figure()
    data = []

    summary_metrics = eval_result.summary_metrics
    if metrics:
        summary_metrics = {
            k: summary_metrics[k]
            for k, v in summary_metrics.items()
            if any(selected_metric in k for selected_metric in metrics)
        }

    data.append(
        go.Bar(
            x=list(summary_metrics.keys()),
            y=list(summary_metrics.values()),
            name=title,
        )
    )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()


def display_radar_plot(eval_results, title: str, metrics=None):
    """Plot the radar plot."""
    fig = go.Figure()
    summary_metrics = eval_results.summary_metrics
    if metrics:
        summary_metrics = {
            k: summary_metrics[k]
            for k, v in summary_metrics.items()
            if any(selected_metric in k for selected_metric in metrics)
        }

    min_val = min(summary_metrics.values())
    max_val = max(summary_metrics.values())

    fig.add_trace(
        go.Scatterpolar(
            r=list(summary_metrics.values()),
            theta=list(summary_metrics.keys()),
            fill="toself",
            name=title,
        )
    )
    fig.update_layout(
        title=title,
        polar=dict(radialaxis=dict(visible=True, range=[min_val, max_val])),
        showlegend=True,
    )
    fig.show()

## 建立 ADK Agent

使用 ADK 建立您的應用程式，包括 Gemini 模型和您定義的自訂工具。

> **重點說明：** 這裡會建立一個產品研究 Agent，具備查詢產品詳細資訊和價格的能力。

### 設定工具

首先，設定客服 Agent 執行工作所需的工具。

> **重點說明：** 這些工具是 Agent 可以呼叫的函式，用於取得產品資訊。

In [ ]:
def get_product_details(product_name: str):
    """收集產品的基本詳細資訊。

    重點：這是一個模擬的工具函式，實際應用中會連接到資料庫或 API
    """
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "usb charger": "A super fast and light usb charger",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """收集產品的價格資訊。

    重點：返回產品價格，如果產品不存在則返回錯誤訊息
    """
    details = {
        "smartphone": 500,
        "usb charger": 10,
        "shoes": 100,
        "headphones": 50,
        "speaker": 80,
    }
    return details.get(product_name, "Product price not found.")

### 設定模型

選擇您的 Agent 將使用的 Gemini AI 模型。如果您想了解 Gemini 及其不同功能，請查看[官方文件](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models)以獲取更多詳細資訊。

> **重點說明：** gemini-2.0-flash 是最新的高效能模型，適合大多數 Agent 應用場景。

In [ ]:
model = "gemini-2.0-flash"

### 組裝 Agent

Vertex AI Gen AI 評估服務可以直接與「可查詢」(Queryable) 的 Agent 一起使用，也可以讓您加入具有特定結構（簽名）的自訂函式。

在這個案例中，您使用自訂函式來組裝 Agent。這個函式會針對給定的輸入觸發 Agent，並解析 Agent 的輸出以提取回應和呼叫的工具。

> **重點說明：** agent_parsed_outcome 函式會執行 Agent 並將結果格式化為評估所需的結構。

In [ ]:
async def agent_parsed_outcome(query):
   # 設定應用程式和會話識別碼
   app_name = "product_research_app"
   user_id = "user_one"
   session_id = "session_one"

# 建立產品研究 Agent
# 重點：instruction 定義了 Agent 的行為邏輯
# 建立產品研究 Agent
# 重點：Agent 是 ADK 的核心元件，負責理解使用者意圖並選擇適當的工具
product_research_agent = Agent(
    name="ProductResearchAgent",  # Agent 的唯一識別名稱
    model=model,  # 使用的 Gemini 模型
    description="一個執行產品研究的 Agent。",  # Agent 的簡短描述
    instruction=f"""
    分析這個使用者請求：'{query}'。
    如果請求是關於價格的，請使用 get_product_price 工具。
    否則，使用 get_product_details 工具來取得產品資訊。
    """,  # 重點：instruction 定義了 Agent 如何選擇和使用工具的邏輯規則
    tools=[get_product_details, get_product_price],  # 重點：提供 Agent 可以呼叫的工具列表
)

   # 建立記憶體內的 session 服務
   # 重點：用於儲存對話歷史和狀態
   session_service = InMemorySessionService()
   await session_service.create_session(
       app_name=app_name, user_id=user_id, session_id=session_id
   )

   # 建立 Runner 來執行 Agent
   runner = Runner(
       agent=product_research_agent, app_name=app_name, session_service=session_service
   )

   # 建立使用者訊息並執行 Agent
   content = types.Content(role="user", parts=[types.Part(text=query)])
   events = [event async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content)]

   # 解析 Agent 輸出為字典格式
   # 重點：提取回應文字和工具呼叫軌跡
   return parse_adk_output_to_dictionary(events)

In [ ]:
# --- 為 Vertex AI 評估提供同步包裝器
# 重點：Vertex AI 評估服務需要同步函式，這裡將非同步函式包裝成同步版本
def agent_parsed_outcome_sync(prompt: str):
    result = asyncio.run(agent_parsed_outcome(prompt))
    # 將 trajectory 轉換為 JSON 字串格式
    result["predicted_trajectory"] = json.dumps(result["predicted_trajectory"])
    return result

### 測試 Agent

查詢您的 Agent。

> **重點說明：** 在進行完整評估之前，先測試 Agent 是否能正常運作。

In [ ]:
# 測試取得產品詳細資訊
# 重點：驗證 Agent 能正確呼叫 get_product_details 工具
response = await agent_parsed_outcome(query="Get product details for shoes")
display(Markdown(format_output_as_markdown(response)))

In [ ]:
# 測試取得產品價格
# 重點：驗證 Agent 能正確呼叫 get_product_price 工具
response = await agent_parsed_outcome(query="Get product price for shoes")
display(Markdown(format_output_as_markdown(response)))

## 使用 Vertex AI Gen AI 評估服務評估 ADK Agent

在使用 AI Agent 時，追蹤其效能和運作情況非常重要。您可以從兩個主要方面來檢視：**監控 (Monitoring)** 和**可觀測性 (Observability)**。

**監控著重於 Agent 執行特定任務的表現：**

* **單一工具選擇**：Agent 是否為任務選擇了正確的工具？

* **多工具選擇（或軌跡）**：Agent 是否在使用工具的順序上做出了合理的選擇？

* **回應生成**：Agent 的輸出是否良好，並且基於其使用的工具是否合理？

**可觀測性著重於了解 Agent 的整體健康狀況：**

* **延遲 (Latency)**：Agent 回應需要多長時間？

* **失敗率 (Failure Rate)**：Agent 無法產生回應的頻率如何？

Vertex AI Gen AI 評估服務可以幫助您在原型設計 Agent 或將其部署到生產環境後評估所有這些方面。它提供[預建的評估標準和指標](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval)，讓您能夠準確了解 Agent 的表現並識別需要改進的領域。

> **重點說明：** 這是一個全面的評估框架，涵蓋從工具選擇到最終輸出品質的所有層面。

### 準備 Agent 評估資料集

要使用 Vertex AI Gen AI 評估服務評估您的 AI Agent，您需要一個特定的資料集，具體取決於您想評估 Agent 的哪些方面。

**資料集應包含：**
- 提供給 Agent 的提示 (prompts)
- 理想或預期的回應（ground truth）【可選】
- 預期的工具呼叫序列（reference trajectory），代表您期望 Agent 為每個給定提示呼叫的工具序列【可選】

> **選擇性提供：** 您也可以提供已生成的回應和預測軌跡（**自備資料集情境 BYOD**）。

下面是一個客服 Agent 資料集的範例，包含使用者提示和參考軌跡。

> **重點說明：** 評估資料集的品質直接影響評估結果的可靠性，建議包含多樣化的測試案例。

In [ ]:
eval_data = {
    "prompt": [
        "Get price for smartphone",
        "Get product details and price for headphones",
        "Get details for usb charger",
        "Get product details and price for shoes",
        "Get product details for speaker?",
    ],
    "predicted_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
}

eval_sample_dataset = pd.DataFrame(eval_data)

列印資料集的一些樣本。

In [ ]:
display_dataframe_rows(eval_sample_dataset, num_rows=3)

### 單一工具使用評估

在設定好您的 AI Agent 和評估資料集後，您就可以開始評估 Agent 是否為給定任務選擇了正確的單一工具。

> **重點說明：** 這是最基本的評估，確認 Agent 能夠識別並選擇正確的工具。

#### 設定單一工具使用指標

Vertex AI Gen AI 評估中的 `trajectory_single_tool_use` 指標為您提供了一種快速方法，用於評估您的 Agent 是否使用了您期望它使用的工具，而不考慮任何特定的工具順序。這是一種基本但有用的方式，可以開始評估在 Agent 處理過程中是否在某個時間點使用了正確的工具。

要使用 `trajectory_single_tool_use` 指標，您需要設定特定使用者請求應該使用哪個工具。例如，如果使用者要求「發送電子郵件」，您可能期望 Agent 使用「send_email」工具，並在使用此指標時指定該工具的名稱。

> **重點說明：** 這個指標只檢查工具是否被使用，不考慮使用順序。

In [ ]:
# 定義單一工具使用指標
# 重點：指定要檢查的工具名稱
single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

#### 執行評估任務

要執行評估，您需要使用預定義的資料集（在此例中為 `eval_sample_dataset`）和指標（`single_tool_usage_metrics`）在實驗中初始化一個 `EvalTask`。然後，使用 agent_parsed_outcome 函式執行評估，並為這個特定的評估執行分配一個唯一識別碼，以儲存和視覺化評估結果。

> **重點說明：** EvalTask 會自動執行 Agent、收集結果並計算指標。

In [ ]:
EXPERIMENT_RUN = f"single-metric-eval-{get_id()}"

single_tool_call_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=single_tool_usage_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/single-metric-eval",
)

single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=agent_parsed_outcome_sync, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(single_tool_call_eval_result)

#### 視覺化評估結果

使用一些輔助函式來視覺化評估結果的樣本。

In [ ]:
display_dataframe_rows(single_tool_call_eval_result.metrics_table, num_rows=3)

### 軌跡評估

在評估 Agent 為給定任務選擇最適當單一工具的能力之後，您可以通過分析相對於使用者輸入的工具序列選擇（軌跡）來泛化評估。這評估了 Agent 是否不僅選擇了正確的工具，而且還以合理且有效的順序使用它們。

> **重點說明：** 軌跡評估比單一工具評估更進階，它檢查工具的使用順序是否合理。

#### 設定軌跡指標

為了評估 Agent 的軌跡，Vertex AI Gen AI 評估提供了幾個基於真實值 (ground-truth) 的指標：

* `trajectory_exact_match`：完全相同的軌跡（相同的動作，相同的順序）

* `trajectory_in_order_match`：參考動作按順序出現在預測軌跡中（允許額外的動作）

* `trajectory_any_order_match`：所有參考動作都出現在預測軌跡中（順序和額外動作不重要）

* `trajectory_precision`：預測動作中存在於參考中的比例

* `trajectory_recall`：參考動作中存在於預測中的比例

所有指標的分數為 0 或 1，除了 `trajectory_precision` 和 `trajectory_recall` 的範圍從 0 到 1。

> **重點說明：** 這些指標提供了不同嚴格程度的軌跡比對方式，可以根據需求選擇合適的指標。

In [ ]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_precision",
    "trajectory_recall",
]

#### 執行評估任務

透過執行新 `EvalTask` 的 `evaluate` 方法來提交評估。

In [ ]:
EXPERIMENT_RUN = f"trajectory-{get_id()}"

trajectory_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=agent_parsed_outcome_sync, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(trajectory_eval_result)

#### 視覺化評估結果

列印並視覺化評估結果的樣本。

In [ ]:
display_dataframe_rows(trajectory_eval_result.metrics_table, num_rows=3)

In [ ]:
# 以長條圖視覺化軌跡指標
# 重點：使用 /mean 後綴來顯示平均值
plot_bar_plot(
    trajectory_eval_result,
    title="Trajectory Metrics",
    metrics=[f"{metric}/mean" for metric in trajectory_metrics],
)

### 評估最終回應

類似於模型評估，您可以使用 Vertex AI Gen AI 評估來評估 Agent 的最終回應。

> **重點說明：** 除了評估工具選擇，還要評估最終輸出的品質。

#### 設定回應指標

在 Agent 推論之後，Vertex AI Gen AI 評估提供了幾種指標來評估生成的回應。您可以使用基於計算的指標來將回應與參考進行比較（如果需要），並使用現有或自訂的基於模型的指標來確定最終回應的品質。

查看[文件](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval)以了解更多資訊。

> **重點說明：** 可以使用預建指標如 safety（安全性）和 coherence（連貫性）來評估回應品質。

In [ ]:
response_metrics = ["safety", "coherence"]

#### 執行評估任務

要評估 Agent 生成的回應，請使用 EvalTask 類別的 `evaluate` 方法。

In [ ]:
EXPERIMENT_RUN = f"response-{get_id()}"

response_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/response-metric-eval",
)

response_eval_result = response_eval_task.evaluate(
    runnable=agent_parsed_outcome_sync, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(response_eval_result)

#### 視覺化評估結果

列印新的評估結果樣本。

In [ ]:
display_dataframe_rows(response_eval_result.metrics_table, num_rows=3)

### 評估基於工具選擇條件的生成回應

在評估與環境互動的 AI Agent 時，標準的文字生成指標（如連貫性）可能不夠充分。這是因為這些指標主要關注文字結構，而 Agent 回應應該根據其在環境中的有效性來評估。

相反，使用自訂指標來評估 Agent 的回應是否在邏輯上遵循其工具選擇，就像您在本節中擁有的那樣。

> **重點說明：** 這個進階評估檢查回應是否合理地反映了 Agent 執行的動作。

#### 定義自訂指標

根據[文件](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval#model-based-metrics)，您可以透過設定此評估的標準和評分系統來定義用於評估 AI Agent 回應是否在邏輯上遵循其動作的提示模板。

定義一個 `criteria` 來設定評估準則，以及一個 `pointwise_rating_rubric` 來提供評分系統（1 或 0）。然後使用 `PointwiseMetricPromptTemplate` 來使用這些元件建立模板。

> **重點說明：** 自訂指標讓您可以根據特定業務邏輯來評估 Agent 行為。

In [ ]:
# 定義評估標準
# 重點：明確定義如何評估回應是否遵循工具執行軌跡
# 定義評估標準
# 重點：明確定義如何評估回應是否遵循工具執行軌跡
criteria = {
    "Follows trajectory": (
        "評估 Agent 的回應是否在邏輯上遵循其執行的動作序列。請考慮以下子要點：\n"
        "  - 回應是否反映了在軌跡中收集到的資訊？\n"
        "  - 回應是否與任務的目標和限制一致？\n"
        "  - 是否存在任何意外或不合邏輯的推理跳躍？\n"
        "請提供來自軌跡和回應的具體範例以支持您的評估。"
    )
}

# 定義評分標準
# 重點：使用二元評分系統（1 = 符合，0 = 不符合）
pointwise_rating_rubric = {
    "1": "Follows trajectory",
    "0": "Does not follow trajectory",
}

# 建立評估提示模板
# 重點：結合標準和評分規則，並指定輸入變數
response_follows_trajectory_prompt_template = PointwiseMetricPromptTemplate(
    criteria=criteria,
    rating_rubric=pointwise_rating_rubric,
    input_variables=["prompt", "predicted_trajectory"],
)

列印此模板的 prompt_data，其中包含組合的標準和評分規則資訊，準備用於評估。

In [ ]:
print(response_follows_trajectory_prompt_template.prompt_data)

在定義了評估提示模板之後，設定相關的指標來評估回應遵循特定軌跡的程度。`PointwiseMetric` 建立一個指標，其中 `response_follows_trajectory` 是指標的名稱，而 `response_follows_trajectory_prompt_template` 提供您之前設定的評估指示或上下文。

> **重點說明：** 這個指標會使用 LLM 來評估回應是否合理地遵循了執行的工具軌跡。

In [ ]:
# 建立自訂的點對點評估指標
# 重點：使用之前定義的模板來建立新的評估指標
response_follows_trajectory_metric = PointwiseMetric(
    metric="response_follows_trajectory",
    metric_prompt_template=response_follows_trajectory_prompt_template,
)

#### 設定回應指標

透過包含自訂指標來設定新的生成回應評估指標。

> **重點說明：** 結合預建指標和自訂指標，獲得全面的評估結果。

In [ ]:
response_tool_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "safety",
    response_follows_trajectory_metric,
]

#### 執行評估任務

執行新的 Agent 評估。

In [ ]:
EXPERIMENT_RUN = f"response-over-tools-{get_id()}"

response_eval_tool_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_tool_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/reasoning-metric-eval",
)

response_eval_tool_result = response_eval_tool_task.evaluate(
    # 如果您提供的是未解析的資料集，請取消以下行的註解
    # 重點：如果資料集包含 prompt 但沒有 predicted_trajectory 和 response，則需要提供 runnable
    #runnable=agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(response_eval_tool_result)

#### 視覺化評估結果

視覺化評估結果樣本。

In [ ]:
display_dataframe_rows(response_eval_tool_result.metrics_table, num_rows=3)

In [ ]:
# 以長條圖視覺化回應指標
# 重點：比較不同指標的表現
plot_bar_plot(
    response_eval_tool_result,
    title="Response Metrics",
    metrics=[f"{metric}/mean" for metric in response_tool_metrics],
)

## 額外內容：自備資料集 (BYOD) 並使用 Vertex AI Gen AI 評估來評估 LangGraph Agent

在自備資料集 (BYOD) [情境](https://cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-dataset)中，您提供 Agent 的預測軌跡和生成的回應。

> **重點說明：** BYOD 適用於您已經有 Agent 執行結果的情況，可以直接評估而不需要重新執行 Agent。

### 自備評估資料集

定義包含預測軌跡和生成回應的評估資料集。

> **重點說明：** BYOD 資料集必須包含 prompt、reference_trajectory、predicted_trajectory 和 response 欄位。

In [ ]:
byod_eval_data = {
    "prompt": [
        "Get price for smartphone",
        "Get product details and price for headphones",
        "Get details for usb charger",
        "Get product details and price for shoes",
        "Get product details for speaker?",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
    "predicted_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
    "response": [
        "500",
        "50",
        "A super fast and light usb charger",
        "100",
        "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    ],
}

byod_eval_sample_dataset = pd.DataFrame(byod_eval_data)
byod_eval_sample_dataset["predicted_trajectory"] = byod_eval_sample_dataset[
    "predicted_trajectory"
].apply(json.dumps)
byod_eval_sample_dataset["reference_trajectory"] = byod_eval_sample_dataset[
    "reference_trajectory"
].apply(json.dumps)
byod_eval_sample_dataset["response"] = byod_eval_sample_dataset["response"].apply(json.dumps)

### 執行評估任務

使用您自己的資料集和與最新評估相同的設定來執行新的 Agent 評估。

> **重點說明：** 使用 BYOD 時不需要提供 runnable 參數，因為資料集已包含所有結果。

In [ ]:
EXPERIMENT_RUN_NAME = f"response-over-tools-byod-{get_id()}"

byod_response_eval_tool_task = EvalTask(
    dataset=byod_eval_sample_dataset,
    metrics=response_tool_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/byod-eval",
)

byod_response_eval_tool_result = byod_response_eval_tool_task.evaluate(
    experiment_run_name=EXPERIMENT_RUN_NAME
)

display_eval_report(byod_response_eval_tool_result)

### 視覺化評估結果

視覺化評估結果樣本。

In [ ]:
display_dataframe_rows(byod_response_eval_tool_result.metrics_table, num_rows=3)

In [ ]:
# 以雷達圖視覺化 ADK Agent 評估結果
# 重點：雷達圖可以同時展示多個指標的表現，便於整體評估
display_radar_plot(
    byod_response_eval_tool_result,
    title="ADK agent evaluation",
    metrics=[f"{metric}/mean" for metric in response_tool_metrics],
)

## 清理資源

> **重點說明：** 執行完評估後，可以選擇刪除實驗資料以節省儲存空間。

In [ ]:
# 設定是否要刪除實驗資料
delete_experiment = True

if delete_experiment:
    try:
        # 刪除實驗及其關聯的 TensorBoard 執行記錄
        # 重點：這會永久刪除所有評估歷史資料
        experiment = aiplatform.Experiment(EXPERIMENT_NAME)
        experiment.delete(delete_backing_tensorboard_runs=True)
    except Exception as e:
        print(e)